## import

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms,datasets, models
from torchvision.models import vgg16,VGG16_Weights
from torch.utils.data import DataLoader, Dataset, Subset

import numpy as np

from sklearn.model_selection import train_test_split

from PIL import Image, ImageFilter
import matplotlib.pyplot as plt

from tqdm import tqdm

import time

## High Parameters

In [2]:
NUM_EPOCHS = 30
LEARNING_RATE = 1e-1
MOMENTUM = 0.9
STEP=5
GAMMA=0.5
EVAL_INTERVAL=1 #验证集测试效果
PATIENTS = 5

In [3]:
transform = transforms.Compose([
    transforms.Resize(size = 32),
    transforms.CenterCrop(size=(32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
dataset = datasets.ImageFolder(root='mydata', transform=transform)
targets = [i[1] for i in dataset.imgs]  # 提取所有图片的标签

# 划分数据集（使用分层抽样）
train_idx, test_idx = train_test_split(range(len(targets)), test_size=0.2, stratify=targets, random_state=42)

# 根据索引创建子集
train_dataset = Subset(dataset, train_idx)
test_dataset = Subset(dataset, test_idx)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## device

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 4, 3)  
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(4, 8, 3)  
        self.fc1 = nn.Linear(8 * 6 * 6, 32)
        self.fc2 = nn.Linear(32, 10)#分类到多少class

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 8 * 6 * 6)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## baseline model

In [6]:
model = ConvNet()
model.to(device)

ConvNet(
  (conv1): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=288, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)

## optimizer

In [7]:
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP, gamma=GAMMA)

## loss function

In [8]:
criterion = nn.CrossEntropyLoss()

In [9]:
def train_batch(model, image, target):
    output = model(image)
    loss = criterion(output,target)
    return output, loss

In [10]:
def test_batch(model, image, target):
    output = model(image)
    loss = criterion(output,target)
    return output, loss

In [11]:
training_loss = []
training_acc = []
testing_loss = []
testing_acc = []



start_time = time.time()
for epoch in range(NUM_EPOCHS):
    model.train()
    torch.cuda.empty_cache()

    ##########################
    ### Training
    ##########################

    running_cls_loss = 0.0
    running_cls_corrects = 0
    
    train_loader = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")


    for batch_idx, (image, target) in enumerate(train_loader):

        image = image.to(device)
        target = target.to(device)

        # train model
        outputs, loss = train_batch(model, image, target)
        _, preds = torch.max(outputs, 1)

        
        loss_data = loss.data.item()
        if np.isnan(loss_data):
            raise ValueError('loss is nan while training')
        running_cls_loss += loss.item()
        running_cls_corrects += torch.sum(preds == target.data)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        train_loader.set_description(f"Epoch {epoch+1}/{NUM_EPOCHS} - Train loss: {running_cls_loss/(batch_idx+1):.4f}")


    epoch_loss = running_cls_loss / len(train_dataset)
    epoch_acc = running_cls_corrects.double() / len(train_dataset)
    training_loss.append(epoch_loss)
    training_acc.append(epoch_acc.cpu().detach().numpy())

    # change learning rate
    scheduler.step()
end_time = time.time()
consume = end_time-start_time
print(training_acc[-1])

        # save the model in last epoch

Epoch 30/30 - Train loss: 1.7349: 100%|██████████| 60/60 [00:03<00:00, 17.07it/s]

0.2133051742344245


In [12]:
model.eval()
    
val_loss = 0.0
val_corrects = 0

for batch_idx, (image, target) in enumerate(test_loader):

    image = image.to(device)
    target = target.to(device)

    # test model
    outputs, loss = test_batch(model, image, target)
    _, preds = torch.max(outputs, 1)

    val_loss += loss.item()
    val_corrects += torch.sum(preds == target.data)
    

val_loss = val_loss / len(test_dataset)
val_acc = val_corrects.double() / len(test_dataset)
                
print(f'Test Acc: {val_acc:.4f}')

Test Acc: 0.2131


In [13]:
print(consume)

111.59760999679565
